<a href="https://colab.research.google.com/github/Vinshul44/Trading-System-with-ML-Automation/blob/main/Ml_model2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
def train_model(df):
    # Create a safe copy to work on
    df1 = df.copy()

    if 'Close' not in df1.columns:
        raise ValueError("Missing 'Close' column")

    # Feature engineering on df1
    df1['Daily_Return'] = df1['Close'].pct_change()
    df1['Return_5D'] = df1['Close'].pct_change(5)
    df1['Volatility_10D'] = df1['Close'].rolling(10).std()

    if 'High' not in df1.columns:
        df1['High'] = df1['Close']
    if 'Low' not in df1.columns:
        df1['Low'] = df1['Close']
    df1['High_Low_Range'] = df1['High'] - df1['Low']
    df1['Target'] = np.where(df1['Close'].shift(-2) > df1['Close'] * 1.005, 1, 0)

    features = [
    'RSI', '20_MA', '50_MA', 'Daily_Return',
    'Return_5D', 'Volatility_10D', 'High_Low_Range', 'Volume'
            ]

    all_cols = features + ['Target']
    df1 = df1.dropna()

    if df1.empty:
        raise ValueError("No valid data to train the model")

    X = df1[features]
    y = df1['Target']

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.2, random_state=42, shuffle=True
    )

    model = DecisionTreeClassifier(max_depth=6, random_state=42)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)

    return model, acc